<a href="https://colab.research.google.com/github/ZerxekLP/BioInformatikColab/blob/main/4_02_GreedySCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shortest Common Superstring Problem
## Greedy Shortest Common Super String

Greedy Algorithms decide at each stage which alternative reduces problem complexity most.

In [2]:
def overlap(a: str, b: str, min_length: int = 3) -> int:
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long. If no such overlap exists, return 0. """
    start = 0  # Start looking for 'b' in 'a'
    while True:
        start = a.find(b[:min_length], start)  # Find occurrence of b's prefix in a
        if start == -1:  # No more occurrences to check
            return 0
        # Check if suffix of a starting at 'start' matches prefix of b
        if b.startswith(a[start:]):
            return len(a) - start
        start += 1


In [4]:
import itertools
from typing import List

def scs(ss: List[str]) -> str:
    """ Returns shortest common superstring of given strings,
        assuming no string is a strict substring of another. """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # Start with the first string in permutation
        for i in range(1, len(ssperm)):
            # Find the overlap between the current superstring and next string
            olen = overlap(sup, ssperm[i], min_length=1)
            # Append the non-overlapping part of the next string to the superstring
            sup += ssperm[i][olen:]
        # Check if this superstring is the shortest one found
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup
    return shortest_sup


In [5]:
from typing import Tuple
def pick_maximal_overlap(reads: List[str], k: int) -> Tuple[str, str, int]:
    """ Return a pair of reads from the list with a
        maximal suffix/prefix overlap >= k. Returns
        overlap length 0 if there are no such overlaps. """
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > best_olen:
            reada, readb = a, b
            best_olen = olen
    return reada, readb, best_olen


In [6]:
from typing import List

def greedy_scs(reads: List[str], k: int) -> str:
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    while True:
        # Find the pair of strings with the maximal overlap
        reada, readb, best_olen = pick_maximal_overlap(reads, k)
        if best_olen == 0:  # No overlap >= k found
            break
        # Merge the two strings with the maximal overlap
        reads.remove(reada)
        reads.remove(readb)
        merged = reada + readb[best_olen:]  # Combine with the overlap removed
        reads.append(merged)  # Add merged string back to reads

    # Join remaining reads into the shortest common superstring
    return ''.join(reads)


In [7]:
%%time
greedy_scs(['ABC', 'BCA', 'CAB'], 2)

CPU times: user 27 µs, sys: 0 ns, total: 27 µs
Wall time: 30.5 µs


'CABCA'

In [8]:
%%time
greedy_scs(['ABCD', 'CDBC', 'BCDA'], 1)

CPU times: user 18 µs, sys: 3 µs, total: 21 µs
Wall time: 23.8 µs


'CDBCABCDA'

In [9]:
%%time
scs(['ABCD', 'CDBC', 'BCDA'])

CPU times: user 25 µs, sys: 0 ns, total: 25 µs
Wall time: 27.9 µs


'ABCDBCDA'